In [1]:
import pandas as pd

### 새로운 댓글 샘플

In [2]:
df = pd.read_excel("text_novelty_detection_sample.xlsx")
df

,comment,author,date,num_likes,title
0,cctv가 버젓이 있는데<br>무슨 생각으로 저랬을까,downy,2023-11-03T02:10:38Z,0,'부산 서면 돌려차기 사건' 드디어 밝혀진 진실 [shorts]
1,35년에서 판사가 용서해줘서 형량 줄어들지 않았나?,ㅇ,2023-11-02T22:06:15Z,0,'부산 서면 돌려차기 사건' 드디어 밝혀진 진실 [shorts]
2,참고로 주먹은 자기자신보다 강한사람을 때리는것 이지요,서종범,2023-11-02T14:04:18Z,0,'부산 서면 돌려차기 사건' 드디어 밝혀진 진실 [shorts]
3,사회에 나와봐야 도움도 안됨/ 사형이 답/ 또 다른 피해자를 만들지 마소서,식물 그리고 커피한잔,2023-11-02T13:02:30Z,0,'부산 서면 돌려차기 사건' 드디어 밝혀진 진실 [shorts]
4,잘 된 일이기는 한데 ...어케 조두순보다 형량이 높지????,프라다임페리얼,2023-11-02T08:31:06Z,0,'부산 서면 돌려차기 사건' 드디어 밝혀진 진실 [shorts]
...,...,...,...,...,...
18173,부모들이 아니라 정확히 &quot;엄마들&quot;<br>선생 괴롭히는거 다 여자들임,ㅇㅇ,2023-09-10T07:47:14Z,8,"[자막뉴스] ""넌 자식이 없어서 그러냐?"" 폭언하던 학부모, 결국... / YTN"
18174,종교인이 우울증 걸린다? ㅋ.ㅋ<br>사이비냐?,아수라발발타,2023-09-10T09:18:39Z,0,"[자막뉴스] ""넌 자식이 없어서 그러냐?"" 폭언하던 학부모, 결국... / YTN"
18175,@아수라발발타너같은 마귀가 득세하면 그럴수도 있는거지.,왜구박멸자,2023-09-11T00:31:13Z,0,"[자막뉴스] ""넌 자식이 없어서 그러냐?"" 폭언하던 학부모, 결국... / YTN"
18176,​​@ㅇㅇ여기서도 갈라치기냐 왕의 dna사건도 아빤데,rdsdhkhcgj,2023-09-11T05:08:13Z,6,"[자막뉴스] ""넌 자식이 없어서 그러냐?"" 폭언하던 학부모, 결국... / YTN"


In [12]:
from bs4 import BeautifulSoup
import emoji
from soynlp.normalizer import repeat_normalize
import re
from emoji import core

In [3]:
!pip install emoji

In [4]:
!pip install soynlp

In [5]:
# 이 아래에는 함수를 적는 곳입니다.
def remove_html_tags(text):
    if isinstance(text, str):
        soup = BeautifulSoup(text, "html.parser")
        clean_text = soup.get_text()
        return clean_text
    else:  # Non-string values (e.g., NaN, None, float)
        return text  # or return "" if you want to return an empty string

def contains_korean(text):
    if not isinstance(text, str):
        return False
    pattern = r"[ㄱ-ㅎㅏ-ㅣ가-힣]"
    return bool(re.search(pattern, text))

def clean_text(inputString):
  text_rmv = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', inputString)
  return text_rmv

In [13]:
def corpus_preprocess(df):
    df = df[~df['comment'].str.contains("@", na=False)]          # 저격 대댓글 걸러냄

    df['comment'] = df['comment'].apply(remove_html_tags)
    df = df[df["comment"].apply(contains_korean)] # 한글 없는 텍스트 제거
    df['comment'] = df['comment'].apply(lambda x: core.replace_emoji(x, replace= "")) # 이모티콘 제거
    df['comment'] = df['comment'].apply(clean_text) # 특수문자 제거
    df['comment'] = df['comment'].apply(lambda x: ' '.join(x.split()))  # 다중 띄어쓰기 -> 단일 띄어쓰기
    df = df.reset_index(drop = True)
    df = df.drop_duplicates(["comment", "author"]) # 중복 댓글 제거
    df = df.reset_index(drop = True)

    df["length"] = df["comment"].str.len() 
    df = df[(df["length"] > 3)&(df["length"] < 500)] # 길이 3 미만 -> 의미없는 댓글이므로 지움

    return df

In [14]:
df = corpus_preprocess(df)
df

C:\Users\hwwe1\AppData\Local\Temp\ipykernel_23312\893606400.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment'] = df['comment'].apply(remove_html_tags)


,comment,author,date,num_likes,title,length
0,cctv가 버젓이 있는데무슨 생각으로 저랬을까,downy,2023-11-03T02:10:38Z,0,'부산 서면 돌려차기 사건' 드디어 밝혀진 진실 [shorts],25
1,35년에서 판사가 용서해줘서 형량 줄어들지 않았나,ㅇ,2023-11-02T22:06:15Z,0,'부산 서면 돌려차기 사건' 드디어 밝혀진 진실 [shorts],27
2,참고로 주먹은 자기자신보다 강한사람을 때리는것 이지요,서종범,2023-11-02T14:04:18Z,0,'부산 서면 돌려차기 사건' 드디어 밝혀진 진실 [shorts],29
3,사회에 나와봐야 도움도 안됨 사형이 답 또 다른 피해자를 만들지 마소서,식물 그리고 커피한잔,2023-11-02T13:02:30Z,0,'부산 서면 돌려차기 사건' 드디어 밝혀진 진실 [shorts],39
4,잘 된 일이기는 한데 어케 조두순보다 형량이 높지,프라다임페리얼,2023-11-02T08:31:06Z,0,'부산 서면 돌려차기 사건' 드디어 밝혀진 진실 [shorts],27
...,...,...,...,...,...,...
17872,자식이 없는걸 지구님은 바라실겁니다 인간의 번식은 가치쓸모가 없는 쓰레기라고 지구님...,E지구에 붙어사는 인간!,2023-09-10T05:17:08Z,1,"[자막뉴스] ""넌 자식이 없어서 그러냐?"" 폭언하던 학부모, 결국... / YTN",203
17873,집에서 당신이 키우세요,카일라스를 꿈꾸며,2023-09-10T05:15:41Z,88,"[자막뉴스] ""넌 자식이 없어서 그러냐?"" 폭언하던 학부모, 결국... / YTN",12
17874,애를 소중히 한다고는 하면서 애 망치는 부모들,김준하,2023-09-10T05:12:01Z,721,"[자막뉴스] ""넌 자식이 없어서 그러냐?"" 폭언하던 학부모, 결국... / YTN",25
17875,부모들이 아니라 정확히 엄마들 선생 괴롭히는거 다 여자들임,ㅇㅇ,2023-09-10T07:47:14Z,8,"[자막뉴스] ""넌 자식이 없어서 그러냐?"" 폭언하던 학부모, 결국... / YTN",32


In [15]:
df.to_excel("text_novelty_detection_sample_preprocessed.xlsx", index = None)

### 사전 수정

In [17]:
import json

In [19]:
with open("C:/Users/hwwe1/Desktop/ytb_tm_files/project_mj/train/filter_words_1.0.json", encoding="utf-8") as f:
    filter_words = json.load(f)
filter_words = [str(word) for word in filter_words]
filter_words = str(filter_words).strip("[]")
print(filter_words)

'예수|하나님|석열|슬롯|문재인|문재앙|이재명|윤석열|민주당|한동훈|장관|법무부|더불어민주당|허경영|슬@롯@머@신|홀??덤|전라도|대깨문|좌파|우파|극우|극좌|이준석|정치|국회의원|선전|선동|사상|검찰|대통령|천공|건희|국민의힘|국힘|운동권|탄핵|정권교체|좌좀|이념|일베|클리앙|남팍|홍어|노무현|달빛창|김대중|땅끄|전두환|보빨|물소|버팔로|이기야|한남|냄져|김치녀|국회|국개|의원|오세훈|검새|완박|새누리|박근혜|우병우|최순실|대남|국정|청문회|박정희|이승만|야당|여당|민노총|최강욱|추미애|윤버버|황교안|간첩|안철수|윤통|자유한국당|미래통합당|보수|진보|기레기|검수완박|법조|공직|1찍|2찍|김동연|윤두창|박형준|이명박|대선|한녀|총선|정치병|정권|장관|유승민|개저씨|몸매'


In [30]:
filter_words_new = '예수|하나님|석열|슬롯|문재인|문재앙|이재명|윤석열|민주당|한동훈|장관|법무부|더불어민주당|허경영|전라도|대깨문|좌파|우파|극우|극좌|이준석|정치|국회의원|선전|선동|사상|검찰|대통령|천공|건희|국민의힘|국힘|운동권|탄핵|정권교체|좌좀|이념|일베|클리앙|남팍|홍어|노무현|김대중|땅끄|전두환|국회|국개|의원|오세훈|검새|완박|새누리|박근혜|우병우|최순실|대남|국정|청문회|박정희|이승만|야당|여당|민노총|최강욱|추미애|윤버버|황교안|간첩|안철수|윤통|자유한국당|미래통합당|보수|진보|검수완박|법조|공직|1찍|2찍|김동연|윤두창|박형준|이명박|대선|총선|정치병|정권|장관|유승민'

In [31]:
filter_words_new

'예수|하나님|석열|슬롯|문재인|문재앙|이재명|윤석열|민주당|한동훈|장관|법무부|더불어민주당|허경영|전라도|대깨문|좌파|우파|극우|극좌|이준석|정치|국회의원|선전|선동|사상|검찰|대통령|천공|건희|국민의힘|국힘|운동권|탄핵|정권교체|좌좀|이념|일베|클리앙|남팍|홍어|노무현|김대중|땅끄|전두환|국회|국개|의원|오세훈|검새|완박|새누리|박근혜|우병우|최순실|대남|국정|청문회|박정희|이승만|야당|여당|민노총|최강욱|추미애|윤버버|황교안|간첩|안철수|윤통|자유한국당|미래통합당|보수|진보|검수완박|법조|공직|1찍|2찍|김동연|윤두창|박형준|이명박|대선|총선|정치병|정권|장관|유승민'

In [32]:
def write_json(data, filename):
    with open(filename, 'w') as file:
        json.dump(data, file)

write_json(filter_words_new, 'political_comments_dict.json')

In [33]:
with open('political_comments_dict.json', encoding="utf-8") as f:
    filter_words = json.load(f)


In [34]:
filter_words

'예수|하나님|석열|슬롯|문재인|문재앙|이재명|윤석열|민주당|한동훈|장관|법무부|더불어민주당|허경영|전라도|대깨문|좌파|우파|극우|극좌|이준석|정치|국회의원|선전|선동|사상|검찰|대통령|천공|건희|국민의힘|국힘|운동권|탄핵|정권교체|좌좀|이념|일베|클리앙|남팍|홍어|노무현|김대중|땅끄|전두환|국회|국개|의원|오세훈|검새|완박|새누리|박근혜|우병우|최순실|대남|국정|청문회|박정희|이승만|야당|여당|민노총|최강욱|추미애|윤버버|황교안|간첩|안철수|윤통|자유한국당|미래통합당|보수|진보|검수완박|법조|공직|1찍|2찍|김동연|윤두창|박형준|이명박|대선|총선|정치병|정권|장관|유승민'